In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 92.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 25.7 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("wumusill/final_project_kogpt2")
model = AutoModelForCausalLM.from_pretrained("wumusill/final_project_kogpt2")

Downloading:   0%|          | 0.00/301 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/123 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

In [3]:
import torch

# GPU 사용 여부
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

model.to(device)
model.eval()
print(device)

cpu


In [130]:
def force_poem(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []
    len_sequence = 0

    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
        # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
        link_with_pre_sentence = " ".join(res_l) + " " + val  
        # print(link_with_pre_sentence)

        # 연결된 문장을 인코딩
        input_ids = tokenizer.encode(
        link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

        # 인코딩 값으로 문장 생성
        output_sequence = model.generate(
            input_ids=input_ids.to(device), 
            do_sample=True,
            max_length=42,
            min_length=len_sequence + 2,
            temperature=0.9,
            repetition_penalty=1.5,
            no_repeat_ngram_size=2)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]

        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        # 사용할 generated_sequence 가 5보다 짧으면 강제적으로 길이를 8로 해준다... 
        if tokenizer.pad_token_id in generated_sequence:
            split_index = generated_sequence.index(tokenizer.pad_token_id)
            split_index = split_index if split_index-len_sequence > 5 else len_sequence + 8
            generated_sequence = generated_sequence[:split_index]

        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        generated_sequence = generated_sequence[len_sequence:]
        # 다음 음절을 위해 길이 갱신
        len_sequence += len([ elem for elem in generated_sequence if elem not in(tokenizer.all_special_ids)])        
        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)


    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [114]:
force_poem("한성민")

한 : 한 번쯤은 돌아봐줘요
성 : 성난 바람 타고 가진 내
민 : 민들레 운명처럼 왜


In [129]:
force_poem("박경택")

박 : 박수를 치자 무서워
경 : 경림아 젖은 비 바람은
택 : 택시처럼 데려다 주는


In [128]:
force_poem("박선영")

박 : 박혀있는 너 땜에
선 : 선물이 그대 삐번
영 : 영영 멈춰주지 않는 걸


In [126]:
force_poem("문종현")

문 : 문득 그대가 왔다는 생각에
종 : 종일 겨운 눈빛
현 : 현 합한삐잊어서 그저 바라만보네요 내 마음이 들리나봐봅니다 그대도 날 잊어주길 바래


In [127]:
force_poem("박조은")

박 : 박혀있는 너 땜에
조 : 조급해져 정말 나와 함께
은 : 은 떠났 크리스마스에는은


In [135]:
def long_line_poem(input_letter):
    # 두음 법칙 사전
    dooeum = {"라":"나", "락":"낙", "란":"난", "랄":"날", "람":"남", "랍":"납", "랑":"낭", 
          "래":"내", "랭":"냉", "냑":"약", "략":"약", "냥":"양", "량":"양", "녀":"여", 
          "려":"여", "녁":"역", "력":"역", "년":"연", "련":"연", "녈":"열", "렬":"열", 
          "념":"염", "렴":"염", "렵":"엽", "녕":"영", "령":"영", "녜":"예", "례":"예", 
          "로":"노", "록":"녹", "론":"논", "롱":"농", "뢰":"뇌", "뇨":"요", "료":"요", 
          "룡":"용", "루":"누", "뉴":"유", "류":"유", "뉵":"육", "륙":"육", "륜":"윤", 
          "률":"율", "륭":"융", "륵":"늑", "름":"늠", "릉":"능", "니":"이", "리":"이", 
          "린":'인', '림':'임', '립':'입'}
    # 결과물을 담을 list
    res_l = []
    len_sequence = 0
    # 한 글자씩 인덱스와 함께 가져옴
    for idx, val in enumerate(input_letter):
        # 두음 법칙 적용
        if val in dooeum.keys():
            val = dooeum[val]

        # 만약 idx 가 0 이라면 == 첫 글자
        if idx == 0:
            # 첫 글자 인코딩
            input_ids = tokenizer.encode(
            val, add_special_tokens=False, return_tensors="pt")

            # 첫 글자 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True,
                max_length=42,
                min_length=5,
                temperature=0.9, 
                repetition_penalty=1.5,
                no_repeat_ngram_size=2
                )
        
        # 첫 글자가 아니라면
        else:
            # 좀더 매끄러운 삼행시를 위해 이전 문장이랑 현재 음절 연결
            # 이후 generate 된 문장에서 이전 문장에 대한 데이터 제거
            link_with_pre_sentence = " ".join(res_l) + " " + val  

            # 연결된 문장을 인코딩
            input_ids = tokenizer.encode(
            link_with_pre_sentence, add_special_tokens=False, return_tensors="pt")

            # 인코딩 값으로 문장 생성
            output_sequence = model.generate(
                input_ids=input_ids.to(device), 
                do_sample=True,
                max_length=52,
                min_length=10,
                temperature=0.9,
                repetition_penalty=1.5,
                no_repeat_ngram_size=2)

        # 생성된 문장 리스트로 변환 (인코딩 되어있고, 생성된 문장 뒤로 padding 이 있는 상태)
        generated_sequence = output_sequence.tolist()[0]
        
        # padding index 앞까지 slicing 함으로써 padding 제거, padding이 없을 수도 있기 때문에 조건문 확인 후 제거
        if tokenizer.pad_token_id in generated_sequence:
            
            # pad token 이 3개 이상이면 세번째에서 자르고, 아니면 첫번째에서 자르고
            pad_index_list = [i for i in range(len(generated_sequence)) if generated_sequence[i] == tokenizer.pad_token_id]
            if len(pad_index_list) > 2:
                split_point = 2
            else : 
                split_point = 0
            # generated_sequence = generated_sequence[:generated_sequence.index(tokenizer.pad_token_id)]
            generated_sequence = generated_sequence[:pad_index_list[split_point]]
        # 첫 글자가 아니라면, generate 된 음절만 결과물 list에 들어갈 수 있게 앞 문장에 대한 인코딩 값 제거
        # print(generated_sequence)
        if idx != 0:
            # 이전 문장의 길이 이후로 슬라이싱해서 앞 문장 제거            
            generated_sequence = generated_sequence[len_sequence:]
            # 다음 음절을 위해 길이 갱신
            len_sequence += len([ elem for elem in generated_sequence if elem not in(tokenizer.all_special_ids)])
        
        # 첫 글자라면
        else:
            # 시퀀스 길이 저장
            len_sequence = len([ elem for elem in generated_sequence if elem not in(tokenizer.all_special_ids)])
        
        # 결과물 디코딩
        decoded_sequence = tokenizer.decode(generated_sequence, clean_up_tokenization_spaces=True, skip_special_tokens=True)

        # 결과물 리스트에 담기
        res_l.append(decoded_sequence)

    # 결과물 list에서 한 줄씩 출력
    for letter, res in zip(input_letter, res_l):
        print(f"{letter} :", res)

In [136]:
long_line_poem("해파리")

해 : 해준 것도 하나 없던 나였었는데
파 : 파 하자릴 먼저리던 너의 모습 사는게 너무 힘들지만 이젠 더는 안되겠어 헤어졌던 그 순간이야 무너지는 나를 왜이기는 하지만
리 : 세상에서


In [137]:
long_line_poem("파이널")

파 : 파랗게 웃던 네 웃음도
이 : 이젠 준 착하고 착한 사람아 어떡해 아무리 생각해도 너만 떠올라
널 : 널 웃게 하는 일 넌 모른 척 해도 날 울린 여자야


In [140]:
long_line_poem("사랑해")

사 : 사려했던화산감 피고못해 바보처럼
랑 : 낭떠꽃같은 내 사랑은
해 : 해피엔딩


In [6]:
long_line_poem("하늘색")

하 : 하염없이 걷고 또 거닐다 떠올라서 그 날이 언제든 서로 다른 틈에서 만나 보고
늘 : 늘 같은 자리에
색 : 색칠 하고


In [5]:
long_line_poem("코코아")

코 : 코끝을 스치는 촉촉한 기분 그래 너와 나 함께 한 순간 모두 다 꿈인걸
코 : 코 끝을 보면 넌 내 늘 그렇듯이 내게 머물러줘 니가 나의 우릴 지켜봐주길 바랄까
아 : 아침을 생각해


In [10]:
long_line_poem("이지혜")

이 : 이 노랠 듣고 다시 내게 돌아와
지 : 지우고 또 비워내도 그대 더는하기 싫어
혜 : 혜야놓지 않을래 이미 늦었어 이젠 다른 사람 만나 행복할 수 있잖아요


In [11]:
long_line_poem("김의준")

김 : 김건모 노래 파란 하늘 아랜 햇살 속에 내 사랑은
의 : 의지로
준 : 준 이후 그리움 스며들어 오면


In [12]:
long_line_poem("데이빗")

데 : 데리러 올 테니 늦잠 자고 나서 어린 날이 다가와서
이 : 이젠 내 품에 안겨 술에 취해
빗 : 빗 갚을 때에도


In [25]:
long_line_poem("구자현")

구 : 구차하게 붙잡았어
자 : 자꾸 듯한 말은 이제 다시 하지 말자 이제는 그만하자 다른 남자와 뭘해
현 : 현기종 날 흔드는 거니 보채지지 않게 해줄게 너무 힘든 약속 꼭 안고서 하는거야 우리크림없이


In [26]:
long_line_poem("문종현")

문 : 문득 지나갈 때면오를때까지 널 사랑했어 지금까지 많은 날을 버려둔채 살았지만
종 : 종일 이렇게 너를 지워볼거야
현 : 현 혹시라도 내게 돌아올까 봐 그게 참 두렵기도해


In [15]:
long_line_poem("권소희")

권 : 권태로 변해 두 눈을 가려도
소 : 소용이 없어
희 : 희미한 기억 저 너머 우리 모습


In [20]:
long_line_poem("최지영")

최 : 최선을 다하면 파란 늦 가을 또 다시 밀었어 너무 바쁜 세상 속에 우연히 널봤지
지 : 지웠을까 우리 서로 스쳐갈때도 있지만
영 : 영원을 태우리 마음을 모두 걸어 눈모아닌 마지막까지 당신을 남겨 혹시게


In [27]:
long_line_poem("한성민")

한 : 한 번만 날 위한 그대가
성 : 성큼 정말 떠나고 싶어 나를 버리 야속하게 가시니 찢어지는 꽃송이마저
민 : 민들레


In [18]:
long_line_poem("우리팀")

우 : 우린 서로 사랑하고 있어
리 : 이젠
팀 : 팀에도 미안함도
